In [22]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pd.options.mode.chained_assignment = None  # default='warn'
import datetime,re,json,pickle

C:\Users\tcobolukbasi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [68]:
def get_insert_query_from_df(df, dest_table):

    insert = """
    INSERT INTO '{dest_table}' (
        """.format(dest_table=dest_table)

    columns_string = str(list(df.columns))[1:-1]
    columns_string = re.sub(r' ', '\n        ', columns_string)
    columns_string = re.sub(r'\'', '', columns_string)

    values_string = ''

    for row in df.itertuples(index=False,name=None):
        values_string += re.sub(r'nan', 'null', str(row))
        values_string += ',\n'

    return insert + columns_string + ')\n     VALUES\n' + values_string[:-2] + ';'


def BaseBand_parser(df_all):
    # Extract Parameters
    TimeStamp = df_all.beginTime.dropna().values[0]
    SubNetwork1 = df_all["dnPrefix"].dropna().values[0].split(",")[0].split("=")[1]
    SubNetwork2 = df_all["dnPrefix"].dropna().values[0].split(",")[1].split("=")[1]
    SubNetwork3 = df_all["dnPrefix"].dropna().values[0].split(",")[2].split("=")[1]
    MeContext = df_all["dnPrefix"].dropna().values[0].split(",")[3].split("=")[1]

    # Extract measInfoId-measObjLdn Table
    df3 = df_all[["measInfoId","jobId","measObjLdn"]].dropna(how="all")
    df3["jobId"] = df3["jobId"].fillna(method="ffill")
    df3["measInfoId"] = df3["measInfoId"].fillna(method="ffill")
    df3 = df3.dropna()
    df3["measInfoId"] = df3["measInfoId"] + ",JobId=" + df3["jobId"]
    df3 = df3.drop(["jobId"],axis=1)

    # Meas Values
    df = df_all[["measInfoId","jobId","measObjLdn","{http://www.3gpp.org/ftp/specs/archive/32_series/32.435#measCollec}r","p"]]
    df = df.dropna(axis=0,how="all")
    df.columns = ["measInfoId","jobId","measObjLdn","value","p"]
    df.loc[df["measObjLdn"].notna(), 'value'] = None
    df["measInfoId"] = df["measInfoId"].fillna(method="ffill")
    df["jobId"] = df["jobId"].fillna(method="ffill")
    df["measObjLdn"] = df["measObjLdn"].fillna(method="ffill")
    df["measInfoId"] = df["measInfoId"] + ",JobId=" + df["jobId"]
    df = df.drop("jobId",axis=1)
    df = df.merge(df3,on=["measInfoId","measObjLdn"],how="inner")


    # Define Meas Types
    df2 = df_all[["measInfoId","jobId","{http://www.3gpp.org/ftp/specs/archive/32_series/32.435#measCollec}measType","p"]]
    df2.columns = ["measInfoId","jobId","measType","p"]
    df2 = df2.dropna(axis=0,how="all")
    df2["measInfoId"] = df2["measInfoId"].fillna(method="ffill")
    df2["jobId"] = df2["jobId"].fillna(method="ffill")
    df2["measType"] = df2["measType"].fillna(method="ffill")
    df2["measInfoId"] = df2["measInfoId"] + ",JobId=" + df2["jobId"]
    df2 = df2.drop("jobId",axis=1)
    df2 = df2.dropna()
    df_meastypes = df2.drop_duplicates(["measInfoId","measType"])

    #Combine All
    df4 = df.merge(df_meastypes,on=["measInfoId","p"],how="left")
    df4 = df4.drop_duplicates()
    df4 = df4[df4.measType.notna()]

    # Seperate all PmGroups
    measInfoIds = df4.measInfoId.unique()

    # Prepare all counters list
    all_counters = []
    for measInfoId in measInfoIds:
        if "USERDEF-All_Scanners" in measInfoId[:1]:
            df5 = df4[df4["measInfoId"]==measInfoId].pivot(index="measObjLdn",columns="measType",values="value")
            df5 = df5.rename_axis("ID", axis=1).reset_index()
            alist = list(df5.columns)
            alist.remove("measObjLdn")
            all_counters = all_counters + alist

    #dtyps = {}
    for measInfoId in measInfoIds:
        #print("\n\n######### ",measInfoId, " #########\n\n")
        df5 = df4[df4["measInfoId"]==measInfoId].pivot(index="measObjLdn",columns="measType",values="value")
        df5 = df5.rename_axis("ID", axis=1).reset_index()

        # Check if other user's counters are in All_scanners. If not put their counters to db only the counters that not in all_scanners
        if "USERDEF-All_Scanners" not in measInfoId:
            blist = list(df5.columns)
            blist.remove("measObjLdn")
            diff_list = list(set(blist)-set(all_counters))
            if len(diff_list)==0:
                continue
            else:
                df5 = df5.drop(list(set(blist) - set(diff_list)),axis=1)
                #print("Fark listesi: ",diff_list)

        df5["TimeStamp"] = TimeStamp
        df5["SubNetwork1"] = SubNetwork1
        df5["SubNetwork2"] = SubNetwork2
        df5["SubNetwork3"] = SubNetwork3
        df5["MeContext"] = MeContext


        s = df5["measObjLdn"].str.split(",")
        dfz = pd.DataFrame(s.tolist())

        fin_df = pd.DataFrame()
        for i in range(len(dfz.columns)):
            f = dfz.iloc[:, i].str.split("=")
            f = f.dropna()
            dfk = pd.DataFrame(np.vstack(f))
            dfk.columns = ["attr","val"]
            dfk["ind"] = range(len(dfk["attr"]))
            dfk = dfk.pivot(columns = "attr",index="ind",values="val").rename_axis(None, axis=1).reset_index().drop("ind",axis=1)
            fin_df = pd.concat([fin_df, dfk], axis=1, join="outer")

        df5 = df5.drop("measObjLdn",axis=1)

        # Convert the date format for mysql db.
        df5 = pd.concat([df5, fin_df], axis=1, join="inner")
        df5["TimeStamp"] = df5["TimeStamp"].str.split("T").str[0] + " " + df5["TimeStamp"].str.split("T").str[1].str.split("+").str[0]
        cols = df5.columns

        # Convert columns to numeric if possible.
        #for c in cols:
        #    try:
        #        df5[c] = pd.to_numeric(df5[c])
        #    except:
        #        pass
        #dtyps[measInfoId] = df5.dtypes.to_dict()
        a_file = open("dtypes_Baseband.pkl", "rb")
        dtyps = pickle.load(a_file)
        df5 = df5.astype(dtype= dtyps[measInfoId],errors="ignore")
            
        table_name = measInfoId.replace("=","-")
        table_name = table_name.replace(",","_")
        table_name = table_name.replace(".","_")

        df5["TimeStamp"] = pd.to_datetime(df5["TimeStamp"])

        df5.to_csv("csv_baseband/"+measInfoId,sep=";")
        # Prepare SQL commands
        #with open("sql_baseband/"+measInfoId, "w") as f:
        with open("sql_baseband/sql_commands.txt", "a") as f:
            f.write(pd.io.sql.get_schema(df5.reset_index(), measInfoId))
        #with open("sql_baseband/"+measInfoId, "a") as f:
        with open("sql_baseband/sql_commands.txt", "a") as f:
            f.write(get_insert_query_from_df(df5.reset_index(), measInfoId))
    return df5
    ## Save
    #a_file = open("dtypes_Baseband.pkl", "wb")
    #pickle.dump(dtyps, a_file)
    #a_file.close()

def DUS_parser(df_all):
    TimeStamp = df_all["cbt"].unique()[0]
    SubNetwork1 = df_all["sn"].dropna()[0].split(",")[0].split("=")[1]
    SubNetwork2 = df_all["sn"].dropna()[0].split(",")[1].split("=")[1]
    MeContext = df_all["sn"].dropna()[0].split(",")[2].split("=")[1]

    df_all = df_all[df_all["neun"].isna()]  
    df_all = df_all[df_all["nedn"].isna()]
    df_all = df_all[df_all["nesw"].isna()]
    df_all = df_all[df_all["mts"].isna()]
    # Create groups
    dfx = df_all[["gp","mt","moid","r","sf"]]

    dfx["group"] = range(len(dfx["gp"]))
    dfx.loc[dfx["gp"].notna() & dfx["mt"].notna(), 'group'] = None
    dfx.loc[dfx["gp"].isna(), 'group'] = None
    dfx["group"] = dfx["group"].fillna(method="ffill")

    dfx = dfx[dfx["sf"].isna()]

    #dtyps = {}
    for group in list(dfx.group.dropna().unique()):

        dfy = dfx[dfx["group"]==group]
        dfy["erase"] = False

        dfy = dfy.drop(["group","erase"],axis=1)
        cols = dfy["mt"].dropna().to_list()
        dfy["erase"] = False
        dfy.loc[dfy["moid"].notna(), 'erase'] = True
        dfy["moid"] = dfy["moid"].fillna(method="ffill")
        dfy = dfy[dfy["erase"]==False]
        if group==dfx.group.unique()[-1]:
            dfy = dfy.iloc[:-1]
        dfy = dfy.iloc[:-1]
        dfy = dfy[dfy["gp"].isna()]
        dfy = dfy[dfy["moid"].notna()]
        dfy = dfy.drop(["mt","erase","gp"],axis=1)
        col_all = []
        for i in dfy["moid"].unique():
            col_all = col_all + cols
        dfy["att"] = col_all    
        dfy = dfy.pivot(index="moid",values="r",columns="att").rename_axis(None, axis=1).reset_index()
        cols = dfy.columns
        dfy["TimeStamp"]=TimeStamp
        dfy["SubNetwork1"]=SubNetwork1
        dfy["SubNetwork2"]=SubNetwork2
        dfy["MeContext"]=MeContext
        dfy = dfy[["TimeStamp","SubNetwork1","SubNetwork2","MeContext"] + cols.to_list()]


        s = dfy["moid"].str.split(",")
        dfz = pd.DataFrame(s.tolist())
        if len(s)== 0:
            continue
        fin_df = pd.DataFrame()
        for i in range(len(dfz.columns)):
            f = dfz.iloc[:, i].str.split("=")
            f = f.dropna()
            dfk = pd.DataFrame(np.vstack(f))
            dfk.columns = ["attr","val"]
            dfk["ind"] = range(len(dfk["attr"]))
            dfk = dfk.pivot(columns = "attr",index="ind",values="val").rename_axis(None, axis=1).reset_index().drop("ind",axis=1)
            fin_df = pd.concat([fin_df, dfk], axis=1, join="outer")

        dfy = dfy.drop("moid",axis=1)
        dfy = pd.concat([dfy, fin_df], axis=1, join="inner")

        dfy["TimeStamp"] = dfy["TimeStamp"].str[0:4]+"-"+dfy["TimeStamp"].str[4:6]+"-"+dfy["TimeStamp"].str[6:8]+" "+dfy["TimeStamp"].str[8:10]+":"+dfy["TimeStamp"].str[10:12]+":"+dfy["TimeStamp"].str[12:14]
        cols = dfy.columns
        
        #for c in cols:
        #    try:
        #        dfy[c] = pd.to_numeric(dfy[c])
        #    except:
        #        pass
        #dtyps[group] = dfy.dtypes.to_dict()
        a_file = open("dtypes_DUS.pkl", "rb")
        dtyps = pickle.load(a_file)
        dfy = dfy.astype(dtype= dtyps[group],errors="ignore")
        
        dfy["TimeStamp"] = pd.to_datetime(dfy["TimeStamp"])

        dfy.to_csv("csv_DUS/"+str(group))
    ## Save
    #a_file = open("dtypes_DUS.pkl", "wb")
    #pickle.dump(dtyps, a_file)
    #a_file.close()

In [80]:
t1 = datetime.datetime.now()

#df_all = pd.read_xml("4_problematic.xml", xpath=".//*")
#df_all = pd.read_xml("4_err.xml", xpath=".//*")
df_all = pd.read_xml("TEST_DUS/5.xml", xpath=".//*")
#df_all = pd.read_xml("sample_DUS_PM.xml", xpath=".//*")

if "ffv" in df_all.columns:
    DUS_parser(df_all)
elif "fileFormatVersion" in df_all.columns:
    df = BaseBand_parser(df_all)
        
t2 = datetime.datetime.now()
print("Script Run Time: ",t2-t1)


Script Run Time:  0:00:04.029616


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ifHCInBroadcastPkts   4 non-null      int64         
 1   ifHCInMulticastPkts   4 non-null      int64         
 2   ifHCInOctets          4 non-null      int64         
 3   ifHCInUcastPkts       4 non-null      int64         
 4   ifHCOutBroadcastPkts  4 non-null      int64         
 5   ifHCOutMulticastPkts  4 non-null      int64         
 6   ifHCOutOctets         4 non-null      int64         
 7   ifHCOutUcastPkts      4 non-null      int64         
 8   ifInDiscards          4 non-null      int64         
 9   ifInUnknownProtos     4 non-null      int64         
 10  ifOutDiscards         4 non-null      int64         
 11  TimeStamp             4 non-null      datetime64[ns]
 12  SubNetwork1           4 non-null      object        
 13  SubNetwork2           4 

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ifHCInBroadcastPkts   4 non-null      int64         
 1   ifHCInMulticastPkts   4 non-null      int64         
 2   ifHCInOctets          4 non-null      int64         
 3   ifHCInUcastPkts       4 non-null      int64         
 4   ifHCOutBroadcastPkts  4 non-null      int64         
 5   ifHCOutMulticastPkts  4 non-null      int64         
 6   ifHCOutOctets         4 non-null      int64         
 7   ifHCOutUcastPkts      4 non-null      int64         
 8   ifInDiscards          4 non-null      int64         
 9   ifInUnknownProtos     4 non-null      int64         
 10  ifOutDiscards         4 non-null      int64         
 11  TimeStamp             4 non-null      datetime64[ns]
 12  SubNetwork1           4 non-null      object        
 13  SubNetwork2           4 

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ifHCInBroadcastPkts   4 non-null      int64         
 1   ifHCInMulticastPkts   4 non-null      int64         
 2   ifHCInOctets          4 non-null      int64         
 3   ifHCInUcastPkts       4 non-null      int64         
 4   ifHCOutBroadcastPkts  4 non-null      int64         
 5   ifHCOutMulticastPkts  4 non-null      int64         
 6   ifHCOutOctets         4 non-null      int64         
 7   ifHCOutUcastPkts      4 non-null      int64         
 8   ifInDiscards          4 non-null      int64         
 9   ifInUnknownProtos     4 non-null      int64         
 10  ifOutDiscards         4 non-null      int64         
 11  TimeStamp             4 non-null      datetime64[ns]
 12  SubNetwork1           4 non-null      object        
 13  SubNetwork2           4 

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ifHCInBroadcastPkts   3 non-null      int64         
 1   ifHCInMulticastPkts   3 non-null      int64         
 2   ifHCInOctets          3 non-null      int64         
 3   ifHCInUcastPkts       3 non-null      int64         
 4   ifHCOutBroadcastPkts  3 non-null      int64         
 5   ifHCOutMulticastPkts  3 non-null      int64         
 6   ifHCOutOctets         3 non-null      int64         
 7   ifHCOutUcastPkts      3 non-null      int64         
 8   ifInDiscards          3 non-null      int64         
 9   ifInUnknownProtos     3 non-null      int64         
 10  ifOutDiscards         3 non-null      int64         
 11  TimeStamp             3 non-null      datetime64[ns]
 12  SubNetwork1           3 non-null      object        
 13  SubNetwork2           3 